 # RESCALING EVERYTHING

In [1]:
%cd '/mnt/scratch-lustre/horlaville/clara_limlam/limCode2020-master_clara_2/'

/fs/lustre/scratch/horlaville/clara_limlam/limCode2020-master_clara_2


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy.units as u # I use astropy units in everything, very helpful for bookkeeping/debugging

from lim import lim # The base intensity mapping object

matplotlib.rcParams.update({'font.size': 18,'figure.figsize':[8,7]}) # Make figures better
#matplotlib.rcParams,update({}'figure.figsize'] = [8,7]
from scipy.ndimage import gaussian_filter
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [3]:
lc_paths = '/home/dongwooc/scratchspace/pprun_hiz_npz/'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(lc_paths) if isfile(join(lc_paths, f))]
onlyfiles.remove('pksc2npz_5313591.out')
onlyfiles.remove('pksc2npz.sh')
for i in range(len(onlyfiles)):
    onlyfiles[i] = lc_paths+onlyfiles[i]

In [55]:
# Initialize LineModel with default model
m = lim()
m_cii = lim('Lichen_v4', doSim = True)

m_cii.update(model_par = {'zdex': 0.4,
 'M0': 1900000000.0,
 'Mmin': 20000000000,
 'alpha_MH1': 0.74,
 'alpha_LCII': 0.024,
 'alpha0': -1.412,
 'gamma0': 0.31,
 'BehrooziFile': 'sfr_reinterp.dat'},
            dnu = 2.8*u.GHz,
            nuObs = 270*u.GHz,
            Delta_nu = 40*u.GHz,
            tobs = 40000*u.h,
            Omega_field = 4*u.deg**2,
            catalogue_file = '/home/dongwooc/scratchspace/pprun_hiz_npz/COMAP_z5.8-7.9_960Mpc_seed_13819.npz')

Input cosmological model does not match simulations
Setting analytic cosmology to match simulation


# Need to rescale signal by 123%, pspec by (123%)² and noisy mocks need to be regenerated because of noise

Need to check that if a given array is stored somewhere, that I can just overwrite it with np.save at that same location with a new array. Use a dummy example to make sure 100% that it works

In [4]:
dum = np.ones((2,3))
dum

array([[1., 1., 1.],
       [1., 1., 1.]])

In [5]:
np.save('/mnt/scratch-lustre/horlaville/dum.npy', dum)

In [6]:
dumm = np.load('/mnt/scratch-lustre/horlaville/dum.npy')

In [8]:
new_dum = 2*dumm

In [9]:
np.save('/mnt/scratch-lustre/horlaville/dum.npy', new_dum)

In [10]:
new_dumm = np.load('/mnt/scratch-lustre/horlaville/dum.npy')

In [11]:
new_dumm

array([[2., 2., 2.],
       [2., 2., 2.]])

It works

In [13]:
sig_scale = 1.23
pspec_scale = (1.23**2)

Personalize noise:

In [63]:
sigm = m_cii.sigma_N # for tobs = 40000h from parameters update above
sigm

<Quantity 3172.81108492 Jy / sr>

In [64]:
def own_noise(m, sig, pix=1.):
    
    '''
    m is the limlam object
    sig is the 3D cube of intensities
    pix is the pixelization blurring desired. default is 1.
    '''
    
    sm_map = gaussian_filter(sig, [pix, pix, 0])
    noise_sigma  = sigm
    noise_map    = np.random.normal(0, noise_sigma.to(u.Jy/u.sr, equivalencies=u.brightness_temperature(m.nuObs)).value, sig.shape)
    # print(noise_sigma.to(u.Jy/u.sr).value)
    sm_noise_map = sm_map + noise_map
    
    return sm_noise_map

In [65]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [66]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_27/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_27/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_27/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [67]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_35/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_35/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_35/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [68]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_312/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_312/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_312/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [69]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_512/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_512/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_512/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [70]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e9-9/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e9-9/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e9-9/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [71]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10-1/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10-1/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10-1/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [72]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-68/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-68/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-68/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [73]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-80/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-80/sig/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-80/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [74]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-020/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-020/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-020/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [75]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-028/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-028/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-028/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [76]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex03/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex03/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex03/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

In [77]:
for i in range(len(onlyfiles)):
    
    sigi = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex05/sig/cii_sig'+str(i)+'.npy')
    
    sigisc = sig_scale*sigi
    newfor = own_noise(m_cii, sigisc)
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex05/cii_sig'+str(i)+'.npy', sigisc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex05/for/for_tobs40kh/cii_for'+str(i)+'.npy', newfor)

pspec:

In [80]:
pspec_scale

1.5129

In [79]:
for i in range(len(onlyfiles)):
    
    pspec270 = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/pspec/cii_pspec'+str(i)+'.npy')
    pspec226 = np.load('/mnt/scratch-lustre/horlaville/nuObs226/zdex04/cii_pspec_nuObs226/cii_pspec'+str(i)+'.npy')
    
    pspec270sc = pspec_scale*pspec270
    pspec226sc = pspec_scale*pspec226
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/pspec/cii_pspec'+str(i)+'.npy', pspec270sc)
    np.save('/mnt/scratch-lustre/horlaville/nuObs226_corr/zdex04/cii_pspec_nuObs226/cii_pspec'+str(i)+'.npy', pspec226sc)

with zdex:

In [81]:
for i in range(len(onlyfiles)):
    
    pspec = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex00/pspec/cii_pspec'+str(i)+'.npy')    
    
    pspecsc = pspec_scale*pspec
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex00/pspec/cii_pspec'+str(i)+'.npy', pspecsc)

In [82]:
for i in range(len(onlyfiles)):
    
    pspec = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex02/pspec/cii_pspec'+str(i)+'.npy')    
    
    pspecsc = pspec_scale*pspec
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex02/pspec/cii_pspec'+str(i)+'.npy', pspecsc)

In [84]:
for i in range(len(onlyfiles)):
    
    pspec = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex06/pspec/cii_pspec'+str(i)+'.npy')    
    
    pspecsc = pspec_scale*pspec
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex06/pspec/cii_pspec'+str(i)+'.npy', pspecsc)

In [85]:
for i in range(len(onlyfiles)):
    
    pspec = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex08/pspec/cii_pspec'+str(i)+'.npy')    
    
    pspecsc = pspec_scale*pspec
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex08/pspec/cii_pspec'+str(i)+'.npy', pspecsc)

In [86]:
for i in range(len(onlyfiles)):
    
    pspec = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex10/pspec/cii_pspec'+str(i)+'.npy')    
    
    pspecsc = pspec_scale*pspec
    
    np.save('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex10/pspec/cii_pspec'+str(i)+'.npy', pspecsc)

stacks:

In [87]:
sig_scale

1.23

In [90]:
for i in range(len(onlyfiles)):
    
    stac = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/stacks/LBGs_z5-9/sig/sig'+str(i)+'.npy', allow_pickle=True)    
    
    stasc = sig_scale*stac
    
    stasc.dump('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/stacks/LBGs_z5-9/sig/sig'+str(i)+'.npy')

In [91]:
for i in range(len(onlyfiles)):
    
    stac = np.load('/mnt/scratch-lustre/horlaville/nuObs270/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/stacks/cii_z6/sig/sig'+str(i)+'.npy', allow_pickle=True)    
    
    stasc = sig_scale*stac
    
    stasc.dump('/mnt/scratch-lustre/horlaville/nuObs270_corr/zdex04/alpha_cii_0-024/alpha_mhi_0-74/Mmin_2-10e10/alpha0_-1_412/gamma0_0_31/stacks/cii_z6/sig/sig'+str(i)+'.npy')